# Dependences

In [1]:
from numba import cuda
import numpy as np
import math
import time

# Functions

In [2]:
def euclidean(p1,p2): 
    return (p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2 

# First Data

In [3]:
position = np.genfromtxt("init.xyz",skip_header=2,usecols = range(1,4))
radius = max([np.sqrt(sum(position[i]**2)) for i in range(0,position.shape[0])])

outline = np.array(
    [1 if euclidean(position[i],[0,0,0]) > (0.95*radius)**2 else 0
        for i in range(0,position.shape[0])
    ]
)
sum(outline)/outline.shape[0]

0.14320987654320988

In [4]:
position=np.concatenate((position+radius, position-radius), axis=0)
position_cu = cuda.to_device(position)
position_cu.shape

(3240, 3)

# last

In [11]:
@cuda.jit
def euclidean(p1,p2): 
    return (p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2 
@cuda.jit
def dist_kernel(position,idx,r_max):
    x,y = cuda.grid(2)   

    d = euclidean(position[y],position[x])
    if d < r_max**2:
        idx[y,x] = y
    else:
        idx[y,x] = 0

In [12]:
idx = cuda.to_device(np.empty(shape=(position_cu.shape[0],position_cu.shape[0]), dtype=np.int32))
idx.shape

(3240, 3240)

In [13]:
idx_sum = cuda.to_device(np.zeros((1,position_cu.shape[0]),dtype=np.int32))
idx_sum.shape

(1, 3240)

In [19]:
threads = (32,32)
blocks = (math.ceil(position.shape[0]/threads[0]),math.ceil(position.shape[0]/threads[1]))
# dist_kernel[threads,blocks](position_cu,idx,3.0)

(102, 102)

In [22]:
dist_kernel

CUDADispatcher(<function dist_kernel at 0x7f31298c4940>)